In [ ]:
create PROCEDURE Usp_Log_Sys_ReplacePlaceholdersInTemplate
  @OrderCode nvarchar(100),
  @Template NVARCHAR(MAX)
AS
BEGIN
    DECLARE @Sql NVARCHAR(MAX);
    DECLARE @ColumnName NVARCHAR(100);
    DECLARE @Value NVARCHAR(500);
    DECLARE @TagName NVARCHAR(200);
    DECLARE cur CURSOR FOR
    SELECT TagName, ColumnFor FROM tb_Log_NotificationTags;

    OPEN cur;

    FETCH NEXT FROM cur INTO @TagName, @ColumnName;

    WHILE @@FETCH_STATUS = 0
    BEGIN
        IF CHARINDEX(@TagName, @Template) > 0
        BEGIN
            SET @Sql = N'SELECT @Value = ' + QUOTENAME(@ColumnName) + ' 
            FROM [dbo].[tb__Log_Order_OrderMaster] WHERE HAWB = @OrderCode';
            EXEC sp_executesql @Sql, N'@OrderCode nvarchar(100), @Value NVARCHAR(500) OUTPUT', @OrderCode, @Value OUTPUT;

            -- Replace the tag in the template if it exists
            SET @Template = REPLACE(@Template,@TagName, ISNULL(@Value, ''));
        END

        FETCH NEXT FROM cur INTO @TagName, @ColumnName;
    END

    CLOSE cur;
    DEALLOCATE cur;

    -- Return the modified template
    SELECT @Template AS ModifiedTemplate;
END;
